In [ ]:
import os 
import sys 
parent_dir = os.path.dirname(os.getcwd())
# sys.path.append("/home/jbc0071/Documents/CANIDS/code")
sys.path.append(parent_dir + "/code")

from model.CANnoloAttackDetector import CANnoloAttackDetector
from data_helpers.CANDataset import CANDataset
from helpers import calculate_feature_vec_length, seperate_attack_loader, calculate_metrics
from dotenv import load_dotenv
# from data_helpers.CANDataLoader import CANDataLoader
import os

load_dotenv()
data_path = os.getenv('DATA_PATH')
dataset = CANDataset(data_path, log_verbosity=1)

config = {
    "batch_size": 32,
    "delta_time_last_msg": {
        "specific_to_can_id": False,
        "records_back": 30
    },
    "delta_time_last_same_aid": {
        "specific_to_can_id": True,
        "records_back": 15
    },
}

ambient_loader, validation_loader, attack_loader = dataset.get_dataloaders(config)

attack_loaders = seperate_attack_loader(attack_loader, config, remove_non_labelled=True)

In [ ]:
unique_can_ids = dataset.get_unique_can_ids()
num_can_ids = len(unique_can_ids)
feature_vec_length = ambient_loader.features_len

# Load model
model_path = 'models/canolo_model_112.pt'

model_config = {
    "embedding_dim": num_can_ids,
    "lstm_units": 128,
    "dense_units": 256,
    "dropout_rate": 0.2,
    "num_embeddings": max(unique_can_ids) + 1, # not sure why + 1 rn but it works
    "feature_vec_length": calculate_feature_vec_length(config)
}

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from detect import detect_wrapper, get_scores_wrapper
import time

num_attack_files = len(attack_loaders)
detectors = []

threshold = 2.80E-06

start_time = time.time()
for _ in range(num_attack_files):
    detector = CANnoloAttackDetector(model_path, threshold, model_config, force_cpu=True)
    detectors.append(detector)

with ProcessPoolExecutor() as executor:
    list_of_filename_scores_and_labels = list(executor.map(get_scores_wrapper, zip(attack_loaders, detectors)))

for loader in attack_loaders:
    loader.reset()
end_time = time.time()

print(f"Finished in {end_time - start_time} seconds")

print(list_of_filename_and_results)
